# Compute reference probabilities with Monte-Carlo

In this example, we perform a reliability benchmark based on a large Monte-Carlo sample. In order to limit the elapsed time, we consider a limited elapsed time for each problem. In order to get the best possible accuracy within this time limit, we set the coefficient of variation to zero.

In [1]:
import openturns as ot
import otbenchmark as otb
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [2]:
problemslist = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(problemslist)
numberOfProblems

26

In [3]:
coefficientOfVariation = 0.0
maximumOuterSampling = int(10 ** 4)
blockSize = 10 ** 1 # 10 ** 3 for real simulations
blockSize

1000

In [4]:
confidenceLevel = 0.95
maximumDurationSeconds = 5.0

In [5]:
model_names = [problemslist[i].getName() for i in range(numberOfProblems)]
metrics = ["PF", "N. function calls", "PMin", "PMax", "C.O.V.", "Digits", "Time (s)"]
resultArray = np.zeros((numberOfProblems, len(metrics)))
for i in tqdm(range(numberOfProblems)):
    startTime = time.time()
    problem = problemslist[i]
    name = problem.getName()
    event = problem.getEvent()
    g = event.getFunction()
    factory = otb.ProbabilitySimulationAlgorithmFactory()
    algo = factory.buildMonteCarlo(problem)
    algo.setMaximumOuterSampling(maximumOuterSampling)
    algo.setBlockSize(blockSize)
    algo.setMaximumCoefficientOfVariation(coefficientOfVariation)
    timer = ot.TimerCallback(maximumDurationSeconds)
    algo.setStopCallback(timer)
    initialNumberOfCall = g.getEvaluationCallsNumber()
    algo.run()
    result = algo.getResult()
    numberOfFunctionEvaluations = g.getEvaluationCallsNumber() - initialNumberOfCall
    computedProbability = result.getProbabilityEstimate()
    confidenceLength = result.getConfidenceLength(confidenceLevel)
    pmin = computedProbability - 0.5 * confidenceLength
    pmax = computedProbability + 0.5 * confidenceLength
    cov = result.getCoefficientOfVariation()
    if cov > 0.0:
        expectedDigits = -np.log10(cov) - 1.0
    else:
        expectedDigits = 0.0
    stopTime = time.time()
    elapsedTime = stopTime - startTime
    resultArray[i][0] = computedProbability
    resultArray[i][1] = numberOfFunctionEvaluations
    resultArray[i][2] = pmin
    resultArray[i][3] = pmax
    resultArray[i][4] = cov
    resultArray[i][5] = expectedDigits
    resultArray[i][6] = elapsedTime

100%|██████████| 26/26 [01:26<00:00,  3.34s/it]


In [6]:
df = pd.DataFrame(resultArray, index=model_names, columns=metrics)
df

,PF,N. function calls,PMin,PMax,C.O.V.,Digits,Time (s)
RP8,7.908277e-04,3576000.0,7.616779e-04,8.199775e-04,0.018806,0.725694,5.000656
RP14,7.729891e-04,9722000.0,7.555105e-04,7.904677e-04,0.011537,0.937915,5.000320
RP22,4.215900e-03,10000000.0,4.175722e-03,4.256078e-03,0.004862,1.313146,2.563903
RP24,2.873300e-03,10000000.0,2.840108e-03,2.906492e-03,0.005894,1.229598,2.334245
RP25,4.200000e-05,10000000.0,3.798135e-05,4.601865e-05,0.048818,0.311417,2.592579
RP28,3.000000e-07,10000000.0,-3.964536e-08,6.396454e-07,0.577639,-0.761656,2.239455
RP31,3.213800e-03,10000000.0,3.178703e-03,3.248897e-03,0.005572,1.253991,2.233063
RP33,2.586500e-03,10000000.0,2.555004e-03,2.617996e-03,0.006213,1.206702,2.841388
RP35,3.482700e-03,10000000.0,3.446169e-03,3.519231e-03,0.005352,1.271499,2.800209
RP38,8.058584e-03,9337000.0,8.001208e-03,8.115961e-03,0.003633,1.439773,5.000573


In [7]:
df.to_csv('reliability_compute_reference_proba.csv')